In [1]:
import smote_variants as sv
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier


model_names = ["RBF SVM", "Decision Tree", "Random Forest", "Neural Net", "LDA", "LogReg", "SVC", "KNN"]
no_of_splits = 5
Groups = {'Group_1': {'ProWSyn': sv.ProWSyn(), 'AND_SMOTE': sv.AND_SMOTE(), 'SMOTE': sv.SMOTE()},
          'Group_2': {'G_SMOTE': sv.G_SMOTE(), 'Random_SMOTE': sv.Random_SMOTE()},
          'Group_3': {'SMOTE_TomekLinks':sv.SMOTE_TomekLinks(proportion=1.0),'VIS_RST':sv.VIS_RST()},
          'Group_4':{'CBSO':sv.CBSO(),'SMOBD':sv.SMOBD(),'A_SUWO':sv.A_SUWO()}}

classifiers = {"RBF SVM":SVC(gamma=2, C=1, max_iter=1000), "Decision Tree":DecisionTreeClassifier(max_depth=5)
                  , "Random Forest":RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)
                  , "Neural Net":MLPClassifier(alpha=1, max_iter=1000), "LDA":LinearDiscriminantAnalysis()
                  , "LogReg":LogisticRegression(), "SVC":SVC(kernel="linear", C=0.025),"KNN":KNeighborsClassifier(n_neighbors=3)}

## setting up directories
raw_data_dir=r'C:\Users\shubh\Desktop\Methods\raw_data'
X_filename=r'X_4_feature_3_sec_Acc_m_+Gyr_m1_Scale.npy'
y_filename=r'y_4_feature_3_sec_Acc_m_+Gyr_m1_Scale.npy'
saving_dir=r'C:\Users\shubh\Desktop\Methods\k_fold_data'
oversampled_data_dir=r'C:\Users\shubh\Desktop\Methods\oversampled_data'
data_shape=16
similarity_score_dir=r'C:\Users\shubh\Desktop\Methods\similarity_score'
oversampled_data_based_on_similarity_dir=r'C:\Users\shubh\Desktop\Methods\oversampled_data_based_on_similarity'
model_results=r'C:\Users\shubh\Desktop\Methods\model_results'
without_smote_results_dir=r'C:\Users\shubh\Desktop\Methods\without smote results'
final_result_dir=r'C:\Users\shubh\Desktop\Methods\final_result'
# user inputs
groups_to_analyse=['Group_1']


In [2]:
import random
import pandas as pd
import numpy as np
from tqdm import tqdm
from scipy.spatial import distance
from sklearn.model_selection import KFold
import os
from config import Config
from sklearn.preprocessing import LabelEncoder
import smote_variants as sv
import glob
from sklearn.metrics import classification_report
from glob import glob
import plotly.express as px


os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(2)
random.seed(2)

In [4]:
X = np.load(os.sep.join([raw_data_dir,X_filename]))
y = np.load(os.sep.join([raw_data_dir,y_filename]))

In [8]:
len(X)

15280

In [13]:
pd.DataFrame(y)[0].value_counts()

Bike       3975
Walking    3890
Bus        3780
Still      3295
Run         340
Name: 0, dtype: int64

In [19]:
# creating kfold data

def create_kfold_data():
    print("inside k fold")
    X = np.load(os.sep.join([raw_data_dir,X_filename]))
    y = np.load(os.sep.join([raw_data_dir,y_filename]))
    skf = KFold(n_splits=no_of_splits)
    count = 1
    try:
        os.mkdir(os.sep.join([saving_dir]))
    except:
        print("directory exists")
    for train_index, test_index in skf.split(X, y):
        X_train, y_train = X[train_index], y[train_index]
        X_test, y_test = X[test_index], y[test_index]
        np.save(os.sep.join([saving_dir,'Fold_{}_X_train.npy'.format(count)]), X_train)
        np.save(os.sep.join([saving_dir,'Fold_{}_y_train.npy'.format(count)]), y_train)
        np.save(os.sep.join([saving_dir,'Fold_{}_X_test.npy'.format(count)]), X_test)
        np.save(os.sep.join([saving_dir,'Fold_{}_y_test.npy'.format(count)]), y_test)
        count += 1
create_kfold_data()

inside k fold
directory exists


In [20]:
def without_smote_results():
    for fold in range(1, no_of_splits + 1):
        train_data = np.load(os.sep.join([saving_dir, 'Fold_{}_X_train.npy'.format(fold)]))
        train_data = train_data.reshape(-1, data_shape)
        train_onehot = np.load(os.sep.join([saving_dir, 'Fold_{}_y_train.npy'.format(fold)]))
        le_encoder = LabelEncoder()
        y_train = le_encoder.fit_transform(train_onehot)
        X_test = np.load(os.sep.join([saving_dir, 'Fold_{}_X_test.npy'.format(fold)]))
        X_test = X_test.reshape(-1, data_shape)
        test_onehot = np.load(os.sep.join([saving_dir, 'Fold_{}_y_test.npy'.format(fold)]))
        le_encoder = LabelEncoder()
        y_test = le_encoder.fit_transform(test_onehot)
        weighted_f1_score = {}
        for clf_name, clf1 in classifiers.items():
            clf1.fit(train_data, y_train)
            pred = clf1.predict(X_test)
            true = y_test
            weighted_f1_score[clf_name] = (classification_report(true, pred, output_dict=True))['weighted avg'][
                'f1-score']
            pd.DataFrame.from_dict(weighted_f1_score, orient='index').to_csv(
                os.sep.join([without_smote_results_dir, 'fold_' + str(fold) + '.csv']))
without_smote_results()

C:\Users\shubh\anaconda3\lib\site-packages\sklearn\svm\_base.py:246: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
C:\Users\shubh\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\shubh\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\shubh\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and 

C:\Users\shubh\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\shubh\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\shubh\anaconda3\lib\site-packages\sklearn\svm\_base.py:246: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
C:\Users\shubh\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-sco

C:\Users\shubh\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\shubh\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
## creating oversampled data
def oversampling_data():
    for i in range(1,no_of_splits+1):
        train_data = np.load(os.sep.join([saving_dir,'Fold_{}_X_train.npy'.format(i)]))
        train_data = train_data.reshape(-1, data_shape)
        train_onehot = np.load(os.sep.join([saving_dir,'Fold_{}_y_train.npy'.format(i)]))
        le_encoder = LabelEncoder()
        y_train = le_encoder.fit_transform(train_onehot)
        analysis_groups = groups_to_analyse
        for analysis in analysis_groups:
            group=Groups[analysis]
            X = []
            Y = []
            try:
                os.makedirs(os.sep.join([oversampled_data_dir, 'fold_' + str(i), analysis]))
            except Exception as e:
                print('Directory already exists')
            for method in group.keys():
                oversampler = sv.MulticlassOversampling(group[method])
                x, y = oversampler.sample(train_data, y_train)
                X.extend(x)
                Y.extend(y)
                x=pd.DataFrame(x)
                x['label']=y
                x.to_csv(os.sep.join([oversampled_data_dir,'fold_'+str(i),analysis,str(method)+'.csv']),index=False)
            X = pd.DataFrame(X)
            X['label'] = Y
            X.to_csv(os.sep.join([oversampled_data_dir,'fold_'+str(i),analysis,str(analysis)+'.csv']),index=False)
oversampling_data()

2020-09-10 23:52:07,300:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2020-09-10 23:52:07,302:INFO:MulticlassOversampling: Sampling minority class with label: 1
2020-09-10 23:52:07,305:INFO:ProWSyn: Running sampling via ('ProWSyn', "{'proportion': 1.0, 'n_neighbors': 5, 'L': 5, 'theta': 1.0, 'n_jobs': 1, 'random_state': None}")


Directory already exists


2020-09-10 23:52:08,258:INFO:MulticlassOversampling: Sampling minority class with label: 3
2020-09-10 23:52:08,259:INFO:ProWSyn: Running sampling via ('ProWSyn', "{'proportion': 0.2564534231200898, 'n_neighbors': 5, 'L': 5, 'theta': 1.0, 'n_jobs': 1, 'random_state': None}")
2020-09-10 23:52:08,978:INFO:MulticlassOversampling: Sampling minority class with label: 4
2020-09-10 23:52:08,981:INFO:ProWSyn: Running sampling via ('ProWSyn', "{'proportion': 0.18083462132921174, 'n_neighbors': 5, 'L': 5, 'theta': 1.0, 'n_jobs': 1, 'random_state': None}")
2020-09-10 23:52:09,849:INFO:MulticlassOversampling: Sampling minority class with label: 2
2020-09-10 23:52:09,851:INFO:ProWSyn: Running sampling via ('ProWSyn', "{'proportion': 0.2483454144342893, 'n_neighbors': 5, 'L': 5, 'theta': 1.0, 'n_jobs': 1, 'random_state': None}")
2020-09-10 23:52:10,589:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2020-09-10 23:52:10,591:INFO:MulticlassOversampling

Directory already exists


2020-09-10 23:52:29,734:INFO:MulticlassOversampling: Sampling minority class with label: 0
2020-09-10 23:52:29,735:INFO:ProWSyn: Running sampling via ('ProWSyn', "{'proportion': 0.13628363087754589, 'n_neighbors': 5, 'L': 5, 'theta': 1.0, 'n_jobs': 1, 'random_state': None}")
2020-09-10 23:52:30,571:INFO:MulticlassOversampling: Sampling minority class with label: 3
2020-09-10 23:52:30,573:INFO:ProWSyn: Running sampling via ('ProWSyn', "{'proportion': 0.12305335716109267, 'n_neighbors': 5, 'L': 5, 'theta': 1.0, 'n_jobs': 1, 'random_state': None}")
2020-09-10 23:52:31,405:INFO:MulticlassOversampling: Sampling minority class with label: 2
2020-09-10 23:52:31,406:INFO:ProWSyn: Running sampling via ('ProWSyn', "{'proportion': 0.23097014925373135, 'n_neighbors': 5, 'L': 5, 'theta': 1.0, 'n_jobs': 1, 'random_state': None}")
2020-09-10 23:52:32,084:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2020-09-10 23:52:32,085:INFO:MulticlassOversampli

Directory already exists


2020-09-10 23:52:50,853:INFO:MulticlassOversampling: Sampling minority class with label: 3
2020-09-10 23:52:50,854:INFO:ProWSyn: Running sampling via ('ProWSyn', "{'proportion': 0.17093023255813952, 'n_neighbors': 5, 'L': 5, 'theta': 1.0, 'n_jobs': 1, 'random_state': None}")
2020-09-10 23:52:51,644:INFO:MulticlassOversampling: Sampling minority class with label: 0
2020-09-10 23:52:51,644:INFO:ProWSyn: Running sampling via ('ProWSyn', "{'proportion': 0.14794455066921605, 'n_neighbors': 5, 'L': 5, 'theta': 1.0, 'n_jobs': 1, 'random_state': None}")
2020-09-10 23:52:52,530:INFO:MulticlassOversampling: Sampling minority class with label: 2
2020-09-10 23:52:52,532:INFO:ProWSyn: Running sampling via ('ProWSyn', "{'proportion': 0.2336080257972053, 'n_neighbors': 5, 'L': 5, 'theta': 1.0, 'n_jobs': 1, 'random_state': None}")
2020-09-10 23:52:53,239:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2020-09-10 23:52:53,241:INFO:MulticlassOversamplin

KeyboardInterrupt: 

In [24]:
def similarity_values():
    analysis_groups = groups_to_analyse
    for fold in range(1, no_of_splits + 1):
        train_data = np.load(os.sep.join([saving_dir, 'Fold_{}_X_train.npy'.format(fold)]))
        train_data = train_data.reshape(-1, data_shape)
        train_onehot = np.load(os.sep.join([saving_dir, 'Fold_{}_y_train.npy'.format(fold)]))
        le_encoder = LabelEncoder()
        y_train = le_encoder.fit_transform(train_onehot)
        orig_data = pd.DataFrame(train_data)
        orig_data['label'] = y_train
        for analysis in analysis_groups:
            for oversampled_file in glob(os.sep.join([oversampled_data_dir,'fold_'+str(fold),analysis,'*.csv'])):
                X=pd.read_csv(oversampled_file)
                for k in X['label'].unique():
                    print('running similarity score calculation for', 'label',k,'for fold',fold,'for group or method',str(os.path.basename(oversampled_file).split('.')[0]))
                    final_similarity = []
                    for i in tqdm(range(len(X.loc[X.label == k, :]))):
                        dist = []
                        for j in range(0, 10):
                            p = random.randint(0, int(len(X.loc[X.label == k, :])))
                            a = list(X.loc[X.label == k, X.columns != 'label'].iloc[p - 1, :])
                            b = list(orig_data.loc[orig_data.label == k, X.columns != 'label'].iloc[j, :])
                            dist.append(distance.euclidean(a, b))
                        a = np.mean(dist)
                        final_similarity.append(a)
                    try:
                        os.makedirs(os.sep.join([similarity_score_dir,'fold_' + str(fold),str(os.path.basename(oversampled_file).split('.')[0])]))
                    except:
                        print('directory exists')
                    pd.DataFrame(final_similarity).to_csv(os.sep.join([similarity_score_dir,'fold_' + str(fold),str(os.path.basename(oversampled_file).split('.')[0]),'label_'+str(k)+'.csv']))
similarity_values()

  0%|                                                                                 | 4/3975 [00:00<02:00, 32.86it/s]

running similarity score calculation for label 3 for fold 1 for group or method AND_SMOTE


  0%|                                                                                 | 4/3975 [00:00<01:51, 35.49it/s]

directory exists
running similarity score calculation for label 4 for fold 1 for group or method AND_SMOTE


  0%|                                                                                 | 4/3975 [00:00<01:57, 33.69it/s]

...

directory exists
running similarity score calculation for label 2 for fold 1 for group or method AND_SMOTE


  0%|                                                                                        | 0/11925 [00:00<?, ?it/s]

directory exists
running similarity score calculation for label 3 for fold 1 for group or method Group_1


  2%|█▌                                                                            | 242/11925 [00:12<10:21, 18.80it/s]


KeyboardInterrupt: 

In [26]:
def oversampled_data_based_on_similarity():
    analysis_groups = groups_to_analyse
    for fold in range(1, no_of_splits + 1):
        for analysis in analysis_groups:
            files_in_group=glob(os.sep.join([oversampled_data_dir, 'fold_' + str(fold), analysis, '*.csv']))
            for oversampled_file in files_in_group:
                X = pd.read_csv(oversampled_file)
                if os.path.basename(oversampled_file).startswith('Group'):
                    oversampled_data_to_consider = X['label'].value_counts().values.min() / len(files_in_group)
                else:
                    oversampled_data_to_consider = X['label'].value_counts().values.min()
                considerable_data = []
                for m in X['label'].unique():
                    label_data = pd.read_csv(os.sep.join([similarity_score_dir, 'fold_' + str(fold),
                                str(os.path.basename(oversampled_file).split('.')[0]), 'label_' + str(m) + '.csv'])).rename(columns={'Unnamed: 0': 'index', '0': 'similarity'}).sort_values(by='similarity')['index'][
                                 0:int(oversampled_data_to_consider)].index
                    label_data = (X.loc[X.label == m, X.columns != 'label'].iloc[label_data, :])
                    label_data['label'] = m
                    considerable_data.append(label_data)
                all_data = pd.concat(considerable_data)
                all_data = all_data.reset_index(drop=True)
                y_train = all_data.pop('label')
                try:
                    os.makedirs(os.sep.join([oversampled_data_based_on_similarity_dir, 'fold_' + str(fold),
                                                str(os.path.basename(oversampled_file).split('.')[0])]))
                except:
                    print('directory available')
                all_data.to_pickle(os.sep.join([oversampled_data_based_on_similarity_dir, 'fold_' + str(fold),
                                                str(os.path.basename(oversampled_file).split('.')[0]),'all_data.pkl']))
                pd.DataFrame(y_train).to_pickle(os.sep.join([oversampled_data_based_on_similarity_dir, 'fold_' + str(fold),
                                                str(os.path.basename(oversampled_file).split('.')[0]), 'y_train.pkl']))
oversampled_data_based_on_similarity()


directory available
directory available
directory available
directory available
directory available
directory available
directory available
directory available
directory available
directory available
directory available
directory available
directory available
directory available
directory available
directory available
directory available
directory available
directory available
directory available


In [ ]:
def model_running():
    analysis_groups = groups_to_analyse
    for fold in range(1, no_of_splits + 1):
        X_test = np.load(os.sep.join([saving_dir,'Fold_{}_X_test.npy'.format(fold)]))
        X_test = X_test.reshape(-1, data_shape)
        test_onehot = np.load(os.sep.join([saving_dir,'Fold_{}_y_test.npy'.format(fold)]))
        le_encoder = LabelEncoder()
        y_test = le_encoder.fit_transform(test_onehot)
        for analysis in analysis_groups:
            weighted_f1_score = {}
            Training_X = pd.read_pickle(
                os.sep.join([oversampled_data_based_on_similarity_dir, 'fold_'+str(fold), analysis, 'all_data.pkl']))
            Training_y = pd.read_pickle(
                os.sep.join([oversampled_data_based_on_similarity_dir, 'fold_'+str(fold), analysis, 'y_train.pkl']))
            for clf_name, clf1 in classifiers.items():
                clf1.fit(Training_X, Training_y)
                pred = clf1.predict(X_test)
                true = y_test
                weighted_f1_score[clf_name] = (classification_report(true, pred, output_dict=True))['weighted avg'][
                    'f1-score']
                try:
                    os.makedirs(os.sep.join([model_results,'fold_'+str(fold)]))
                except:
                    print('directory exists')
                pd.DataFrame.from_dict(weighted_f1_score,orient='index').to_csv(os.sep.join([model_results,'fold_'+str(fold),analysis+'.csv']))
            for method in Groups[analysis].keys():
                weighted_f1_score={}
                Training_X=pd.read_pickle(os.sep.join([oversampled_data_based_on_similarity_dir,'fold_'+str(fold),method,'all_data.pkl']))
                Training_y=pd.read_pickle(os.sep.join([oversampled_data_based_on_similarity_dir,'fold_'+str(fold),method,'y_train.pkl']))
                for clf_name,clf1 in classifiers.items():
                    clf1.fit(Training_X, Training_y)
                    pred = clf1.predict(X_test)
                    true = y_test
                    weighted_f1_score[clf_name]=(classification_report(true, pred, output_dict=True))['weighted avg']['f1-score']
                    pd.DataFrame.from_dict(weighted_f1_score, orient='index').to_csv(
                        os.sep.join([model_results, 'fold_' + str(fold), method + '.csv']))


In [28]:
def aggregated_results():
    all_dataframe=[]
    for file in glob(os.sep.join([model_results,'*','*.csv'])):
        dataframe=pd.read_csv(file)
        dataframe=dataframe.reset_index(drop=True).rename(columns={'Unnamed: 0':'model_name','0':'probablity'})
        dataframe['name']=os.path.basename(file).split('.')[0]
        all_dataframe.append(dataframe)
    for file in glob(os.sep.join([without_smote_results_dir,'*.csv'])):
        dataframe=pd.read_csv(file)
        dataframe=dataframe.reset_index(drop=True).rename(columns={'Unnamed: 0':'model_name','0':'probablity'})
        dataframe['name'] ='Without smote'
        all_dataframe.append(dataframe)
    final_results=pd.concat(all_dataframe)
    final_results=final_results.groupby(['name','model_name'])['probablity'].mean().reset_index()
    final_results.to_csv(os.sep.join([final_result_dir,'final_results.csv']),index=False)
aggregated_results()

In [29]:
def plotting():
    result=pd.read_csv(os.sep.join([final_result_dir,'final_results.csv']))
    fig = px.line(result, x="model_name", y="probablity", color='name', title='weighted_f1_score comparison')
    fig.show()
plotting()